In [424]:
import pandas as pd

# Load the data
data = pd.read_csv('pose-coordinates/combined_exercise_data.csv')

# Show the first few rows of the data
data.head()


,frame_time,left_hip,left_knee,left_ankle,left_shoulder,left_elbow,left_wrist,right_hip,right_knee,right_ankle,right_shoulder,right_elbow,right_wrist,exercise
0,14:42:21.507552,"[0.5181276798248291, 0.4100392758846283]","[0.5391181707382202, 0.6699928641319275]","[0.5533620715141296, 0.8769230842590332]","[0.5310813188552856, 0.09941217303276062]","[0.5659307241439819, 0.24480246007442474]","[0.5099042654037476, 0.2083798050880432]","[0.4429663419723511, 0.413503497838974]","[0.4267920255661011, 0.6707853078842163]","[0.41609427332878113, 0.8784694075584412]","[0.4191204309463501, 0.10457929968833923]","[0.3930659294128418, 0.265164315700531]","[0.4320838153362274, 0.22246700525283813]",squat
1,14:42:21.576062,"[0.5191106796264648, 0.40894660353660583]","[0.5385134220123291, 0.6662384867668152]","[0.5522972941398621, 0.8755378723144531]","[0.5311716198921204, 0.09929359704256058]","[0.5666308999061584, 0.24667562544345856]","[0.5210150480270386, 0.20836390554904938]","[0.4432834982872009, 0.4135110676288605]","[0.4277413785457611, 0.6697316765785217]","[0.4161878526210785, 0.877396285533905]","[0.4192013442516327, 0.1045059934258461]","[0.39241600036621094, 0.26587486267089844]","[0.42323070764541626, 0.22764305770397186]",squat
2,14:42:21.635898,"[0.5196697115898132, 0.40803226828575134]","[0.5382950901985168, 0.6652030348777771]","[0.5514864325523376, 0.873742938041687]","[0.5309719443321228, 0.09930171072483063]","[0.567449688911438, 0.2480478137731552]","[0.5300108194351196, 0.20709922909736633]","[0.44356223940849304, 0.41347429156303406]","[0.42839792370796204, 0.6683784127235413]","[0.4163113534450531, 0.8769707679748535]","[0.4191102981567383, 0.10445953905582428]","[0.3916473388671875, 0.2716202735900879]","[0.4180949032306671, 0.22782550752162933]",squat
3,14:42:21.694788,"[0.5202763676643372, 0.4086938202381134]","[0.5381088256835938, 0.6649082899093628]","[0.5506188273429871, 0.8735244274139404]","[0.5304761528968811, 0.09899391978979111]","[0.567510724067688, 0.24583746492862701]","[0.5329428911209106, 0.20729772746562958]","[0.44360440969467163, 0.4147385358810425]","[0.42885011434555054, 0.6674184203147888]","[0.4163488447666168, 0.8772566318511963]","[0.419181227684021, 0.10328248143196106]","[0.3914782404899597, 0.2686671018600464]","[0.4191594421863556, 0.22835099697113037]",squat
4,14:42:21.753531,"[0.5203924775123596, 0.40899017453193665]","[0.5382625460624695, 0.6640178561210632]","[0.5506567358970642, 0.8736501932144165]","[0.5304253697395325, 0.09928525984287262]","[0.5675113201141357, 0.24878033995628357]","[0.5339441895484924, 0.20284798741340637]","[0.44352272152900696, 0.416412889957428]","[0.42884352803230286, 0.667438805103302]","[0.41628405451774597, 0.8761215806007385]","[0.41900888085365295, 0.10339131206274033]","[0.38929423689842224, 0.27037400007247925]","[0.41947320103645325, 0.22835388779640198]",squat


In [425]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from ast import literal_eval

# List of body parts
body_parts = ['left_hip', 'left_knee', 'left_ankle', 'left_shoulder', 'left_elbow', 'left_wrist', 
              'right_hip', 'right_knee', 'right_ankle', 'right_shoulder', 'right_elbow', 'right_wrist']

# Convert the string representation of list to actual list for each body part
for body_part in body_parts:
    data[body_part] = data[body_part].apply(literal_eval)

# Split the coordinates into separate columns
for body_part in body_parts:
    data[[f'{body_part}_x', f'{body_part}_y']] = pd.DataFrame(data[body_part].to_list(), index= data.index)

# Drop the original columns
data.drop(body_parts, axis=1, inplace=True)

# Convert the labels to numeric form
le = LabelEncoder()
data['exercise'] = le.fit_transform(data['exercise'])

data.head()


,frame_time,exercise,left_hip_x,left_hip_y,left_knee_x,left_knee_y,left_ankle_x,left_ankle_y,left_shoulder_x,left_shoulder_y,...,right_knee_x,right_knee_y,right_ankle_x,right_ankle_y,right_shoulder_x,right_shoulder_y,right_elbow_x,right_elbow_y,right_wrist_x,right_wrist_y
0,14:42:21.507552,1,0.518128,0.410039,0.539118,0.669993,0.553362,0.876923,0.531081,0.099412,...,0.426792,0.670785,0.416094,0.878469,0.419120,0.104579,0.393066,0.265164,0.432084,0.222467
1,14:42:21.576062,1,0.519111,0.408947,0.538513,0.666238,0.552297,0.875538,0.531172,0.099294,...,0.427741,0.669732,0.416188,0.877396,0.419201,0.104506,0.392416,0.265875,0.423231,0.227643
2,14:42:21.635898,1,0.519670,0.408032,0.538295,0.665203,0.551486,0.873743,0.530972,0.099302,...,0.428398,0.668378,0.416311,0.876971,0.419110,0.104460,0.391647,0.271620,0.418095,0.227826
3,14:42:21.694788,1,0.520276,0.408694,0.538109,0.664908,0.550619,0.873524,0.530476,0.098994,...,0.428850,0.667418,0.416349,0.877257,0.419181,0.103282,0.391478,0.268667,0.419159,0.228351
4,14:42:21.753531,1,0.520392,0.408990,0.538263,0.664018,0.550657,0.873650,0.530425,0.099285,...,0.428844,0.667439,0.416284,0.876122,0.419009,0.103391,0.389294,0.270374,0.419473,0.228354


In [426]:
from random import randint
from sklearn.model_selection import train_test_split

# Constants
n_samples = 20
n_time_steps = 75
n_features = len(body_parts) * 2  # x and y coordinates for each body part

# Reshape the data
X = data.drop(['frame_time', 'exercise'], axis=1).to_numpy().reshape(n_samples, n_time_steps, n_features)
y = data['exercise'].to_numpy()[::n_time_steps]  # Only select the first label of each video

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=randint(1,100))

print("y_train", y_train)

X_train.shape, y_train.shape, X_test.shape, y_test.shape


y_train [0 0 0 0 0 1 0 0 1 1 1 0 1 1 0]


((15, 75, 24), (15,), (5, 75, 24), (5,))

In [427]:
import torch
import torch.nn as nn
import torch.optim as optim

# Check if GPU is available and if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout_rate):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.batchnorm(out[:, -1, :])
        out = self.dropout(out)
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

# Define the model
model = LSTMModel(n_features, 32, 1, 2, 0.2).to(device)

# Define the loss function and the optimizer
criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

# Train the model
epochs = 75
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.view(-1, 1))
    
    # Backward and optimize
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 5 == 0:
        # Print loss
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, epochs, loss.item()))

# Evaluate the model
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor)

# # Apply a threshold of 0.5 to the predictions
# y_test_pred = (y_test_pred > 0.5).float()

# # Compute the accuracy
# accuracy = (y_test_pred.squeeze() == y_test_tensor).float().mean()
# print("Test Accuracy: {:.2f}".format(accuracy.item()))


Epoch [5/75], Loss: 0.6784
Epoch [10/75], Loss: 0.2947
Epoch [15/75], Loss: 0.1784
Epoch [20/75], Loss: 0.1131
Epoch [25/75], Loss: 0.0785
Epoch [30/75], Loss: 0.0520
Epoch [35/75], Loss: 0.0517
Epoch [40/75], Loss: 0.0374
Epoch [45/75], Loss: 0.0278
Epoch [50/75], Loss: 0.0215
Epoch [55/75], Loss: 0.0220
Epoch [60/75], Loss: 0.0175
Epoch [65/75], Loss: 0.0179
Epoch [70/75], Loss: 0.0168
Epoch [75/75], Loss: 0.0149


In [428]:
# Convert y_test_pred tensor to a numpy array
y_test_pred = y_test_pred.numpy()

# Apply threshold of 0.5 to convert values to 1 or 0
y_test_pred_rounded = np.where(y_test_pred > 0.5, 1, 0)

# Flatten the y_test_pred array
y_test_pred_rounded = y_test_pred_rounded.flatten()

# Compare predicted values with actual values
accuracy = np.mean(y_test_pred_rounded == y_test)

print("y_test_pred", y_test_pred)
print("y_test", y_test)
print("Accuracy: %", accuracy * 100)


y_test_pred [[0.9939115 ]
 [0.8720199 ]
 [0.00871343]
 [0.99051887]
 [0.9941896 ]]
y_test [1 1 0 1 1]
Accuracy: % 100.0


In [429]:
# # Save the model
# torch.save(model.state_dict(), 'exercise_classification_model.pth')